In [17]:
from google.colab import files
import pandas as pd
import os

In [2]:
uploaded = files.upload()

Saving bc_veh4223_230215.csv to bc_veh4223_230215.csv


In [18]:
vehicle_df = pd.read_csv("bc_veh4223_230215.csv")
vehicle_df.head()

,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP
0,259173279,259173350,15FEB2023:00:00:00,4223,194493,71279,-122.679572,45.517122,8,1.6
1,259173279,259173350,15FEB2023:00:00:00,4223,194525,71284,-122.679408,45.517392,8,1.9
2,259173279,259173350,15FEB2023:00:00:00,4223,194562,71290,-122.679227,45.517700,8,2.2
3,259173279,259173350,15FEB2023:00:00:00,4223,194598,71295,-122.679048,45.517998,9,0.9
4,259173279,259173350,15FEB2023:00:00:00,4223,194628,71300,-122.678888,45.518255,9,1.3


In [19]:
vehicle_df['OPD_DATE'] = pd.to_datetime(vehicle_df['OPD_DATE'], format='%d%b%Y:%H:%M:%S')
vehicle_df['ACT_TIME'] = pd.to_timedelta(vehicle_df['ACT_TIME'], unit='s')

vehicle_df['TIMESTAMP'] = vehicle_df['OPD_DATE'] + vehicle_df['ACT_TIME']

print("\nThe Data Frame for Feb 15th 2023 for the vehicle - 4223 after adding the Time Stamp column :")
vehicle_df.head()


The Data Frame for Feb 15th 2023 for the vehicle - 4223 after adding the Time Stamp column :


,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP,TIMESTAMP
0,259173279,259173350,2023-02-15,4223,194493,0 days 19:47:59,-122.679572,45.517122,8,1.6,2023-02-15 19:47:59
1,259173279,259173350,2023-02-15,4223,194525,0 days 19:48:04,-122.679408,45.517392,8,1.9,2023-02-15 19:48:04
2,259173279,259173350,2023-02-15,4223,194562,0 days 19:48:10,-122.679227,45.517700,8,2.2,2023-02-15 19:48:10
3,259173279,259173350,2023-02-15,4223,194598,0 days 19:48:15,-122.679048,45.517998,9,0.9,2023-02-15 19:48:15
4,259173279,259173350,2023-02-15,4223,194628,0 days 19:48:20,-122.678888,45.518255,9,1.3,2023-02-15 19:48:20


In [20]:
vehicle_df['dMETERS'] = vehicle_df['METERS'].diff()
vehicle_df['dTIMESTAMP'] = vehicle_df['TIMESTAMP'].diff().dt.total_seconds()

vehicle_df['SPEED'] = vehicle_df.apply( lambda row: row['dMETERS'] / row['dTIMESTAMP'] if pd.notna(row['dTIMESTAMP']) else None, axis=1 )

print("\nThe Data Frame for Feb 15th 2023 for the vehicle - 4223 after adding the Speed column : ")
vehicle_df.head()


The Data Frame for Feb 15th 2023 for the vehicle - 4223 after adding the Speed column : 


,EVENT_NO_TRIP,EVENT_NO_STOP,OPD_DATE,VEHICLE_ID,METERS,ACT_TIME,GPS_LONGITUDE,GPS_LATITUDE,GPS_SATELLITES,GPS_HDOP,TIMESTAMP,dMETERS,dTIMESTAMP,SPEED
0,259173279,259173350,2023-02-15,4223,194493,0 days 19:47:59,-122.679572,45.517122,8,1.6,2023-02-15 19:47:59,NaN,NaN,NaN
1,259173279,259173350,2023-02-15,4223,194525,0 days 19:48:04,-122.679408,45.517392,8,1.9,2023-02-15 19:48:04,32.0,5.0,6.400000
2,259173279,259173350,2023-02-15,4223,194562,0 days 19:48:10,-122.679227,45.517700,8,2.2,2023-02-15 19:48:10,37.0,6.0,6.166667
3,259173279,259173350,2023-02-15,4223,194598,0 days 19:48:15,-122.679048,45.517998,9,0.9,2023-02-15 19:48:15,36.0,5.0,7.200000
4,259173279,259173350,2023-02-15,4223,194628,0 days 19:48:20,-122.678888,45.518255,9,1.3,2023-02-15 19:48:20,30.0,5.0,6.000000


In [21]:
trip_status = vehicle_df.groupby('EVENT_NO_TRIP')['SPEED'].agg(['max', 'median'])

max_trip = trip_status.loc[trip_status['max'].idxmax()]
max_speed = max_trip['max']
max_speed_record = vehicle_df[vehicle_df['SPEED'] == max_speed].iloc[0]

print(f"\n The minimum speed for Feb 15th 2023 for the vehicle - 4223 : {max_speed} m/s" )
print(f"\n The Location ,Longitude, Latitude and time of max speed for Feb 15th 2023 for the vehicle - 4223 : ", max_speed_record['GPS_LONGITUDE'], max_speed_record['GPS_LATITUDE'], max_speed_record['TIMESTAMP'])
print(f"\n The median speed for Feb 15th 2023 for the vehicle - 4223 : {trip_status['median'].median()} m/s")


 The minimum speed for Feb 15th 2023 for the vehicle - 4223 : 17.4 m/s

 The Location ,Longitude, Latitude and time of max speed for Feb 15th 2023 for the vehicle - 4223 :  -122.660822 45.505452 2023-02-15 05:44:49

 The median speed for Feb 15th 2023 for the vehicle - 4223 : 7.2 m/s
